### This code gets bond orders for epoxies by targeting bonds involving oxygen.

I'm not entirely sure if this is the output that we want, so I'm just keeping this here for epoxy1. If this code produces ideal results, I will duplicate the kernel for the other epoxies as well.

In [1]:
import os

import re



def extract_bond_info(directory, filename="freq.out"):
    filepath = os.path.join(directory, filename)

    if not os.path.isfile(filepath):
        print(f"Error: {filepath} does not exist.")
        return None

    with open(filepath, "r") as f:
        lines = f.readlines()

    try:
        start_index = lines.index("  Mayer bond orders larger than 0.100000\n") + 1
    except ValueError:
        print(f"No bond order section in {filepath}")
        return None

    relevant_bonds = []

    for line in lines[start_index:]:
        if not line.strip():
            break

        bonds = line.split("B(")[1:]
        
### These were loops that were previously in the code and added by ChatGPT before adjusting it to avoid errors for bonds that aren't found
#         for bond in bonds:
#             elements = re.findall(r"\d+-(\w+)", bond)
#             order = re.findall(r":\s*([\d.]+)", bond)[0]
#             if any(elem in elements for elem in ["C", "H", "O"]):
#                 relevant_bonds.append(float(order))
        
#         for bond in bonds:
#             elements = re.findall(r"\d+-(\w+)", bond)
#             order = re.findall(r":\s*([\d.]+)", bond)[0]
#             if "O" in elements:
#                 relevant_bonds.append(float(order))
                
        for bond in bonds:
            elements = re.findall(r"\d+-(\w+)", bond)
            order_matches = re.findall(r":\s*([\d.]+)", bond)
            if order_matches:
                order = order_matches[0]
            if "O" in elements:
                relevant_bonds.append(float(order))



    if relevant_bonds:
        average = sum(relevant_bonds) / len(relevant_bonds)
        subdirectory = os.path.basename(directory)
        print(f"\n{subdirectory}\n{'-'*20}\nRelevant bonds\n{'-'*20}")
        for bond in relevant_bonds:
            print(bond)
        print(f"\nAverage = {average:.4f}")
        return subdirectory, average
    else:
        print(f"No relevant bonds in {filepath}")
        return None


def main():
    base_dir = r"C:\Users\sling\OneDrive\Desktop\einr\epoxy1"  # Updated with the path to the epoxy folder on my computer
    averages = {}
    for root, dirs, files in os.walk(base_dir):
        result = extract_bond_info(root)
        if result is not None:
            subdirectory, average = result
            averages[subdirectory] = average

    print("\nSUMMARY\n--------------")
    for subdirectory, average in sorted(averages.items()):
        print(f"{subdirectory}: {average:.4f}")


if __name__ == "__main__":
    main()



epoxy1
--------------------
Relevant bonds
--------------------
0.8714
0.9233

Average = 0.8973

SUMMARY
--------------
epoxy1: 0.8973


***

### This code gets optimization energies for epoxies by using the grep command.

This code fails in various aspects and areas, but I'd like to revisit it in the future. Perhaps I can get some support from Josh or ask ChatGPT to optimize this code and analyze the error messages.

In [2]:
import os

import subprocess



result_dict = {}



for dirpath, dirnames, filenames in os.walk(r"C:\Users\sling\OneDrive\Desktop\einr\epoxy1"):

    for filename in filenames:

        if filename == 'OPT.out':

            full_path = os.path.join(dirpath, filename)

            key = os.path.basename(os.path.dirname(full_path))



            # Use subprocess to execute the tac and grep commands

            tac = subprocess.Popen(['tac', full_path], stdout=subprocess.PIPE)

            grep = subprocess.Popen(['grep', '-m', '1', 'FINAL'], stdin=tac.stdout, stdout=subprocess.PIPE)

            tac.stdout.close()

            output = grep.communicate()[0]

            tac.wait()



            if output:

                # Strip the trailing newline and split on newline to get last result

                last_result = output.decode().strip().split('\n')[-1]

                result_dict[key] = last_result



# Sort the results by key and print them

for key in sorted(result_dict):

    print(key)

    print('-----')

    print(result_dict[key])

    print()

FileNotFoundError: [WinError 2] The system cannot find the file specified